In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import os


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# load data (currently hosted on github!)
!git clone https://github.com/nadakhn/cds-project.git

Cloning into 'cds-project'...
remote: Enumerating objects: 24074, done.
remote: Total 24074 (delta 0), reused 0 (delta 0), pack-reused 24074
Receiving objects: 100% (24074/24074), 234.61 MiB | 13.52 MiB/s, done.
Resolving deltas: 100% (12/12), done.
Updating files: 100% (27792/27792), done.


In [4]:
class AgePredictionDataset(Dataset):
    def __init__(self, csv_file, img_dir, processor):
        self.data_frame = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.processor = processor

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.data_frame.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        age = self.data_frame.iloc[idx, 1]
        inputs = self.processor(images=image, return_tensors="pt")
        return inputs.pixel_values.squeeze(), age


In [5]:
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

train_dataset = AgePredictionDataset(
    csv_file='/content/cds-project/image/sampled_data/train_data.csv',
    img_dir='/content/cds-project/image/sampled_data/train_data',
    processor=processor)

# Assuming 'val_data.csv' and 'test_data.csv' exist and 'val_data' and 'test_data' directories also exist
val_dataset = AgePredictionDataset(
    csv_file='/content/cds-project/image/sampled_data/val_data.csv',
    img_dir='/content/cds-project/image/sampled_data/val_data',
    processor=processor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

In [6]:
class CLIPForAgePrediction(nn.Module):
    def __init__(self):
        super(CLIPForAgePrediction, self).__init__()
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.regression_head = nn.Linear(self.clip_model.config.projection_dim, 1)

    def forward(self, pixel_values):
        image_features = self.clip_model.get_image_features(pixel_values)
        age = self.regression_head(image_features)
        return age

model = CLIPForAgePrediction().to(device)


pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [7]:
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)


In [8]:
for epoch in range(30):
    model.train()
    running_loss = 0.0
    for inputs, ages in train_loader:
        inputs, ages = inputs.to(device), ages.to(device).float()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), ages)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')



Epoch 1, Loss: 499.29643417603194
Epoch 2, Loss: 486.91961479798994
Epoch 3, Loss: 486.775826512301
Epoch 4, Loss: 485.0397426217635
Epoch 5, Loss: 482.541164190119
Epoch 6, Loss: 469.32537716952237
Epoch 7, Loss: 465.2032279641871
Epoch 8, Loss: 460.7653578447148
Epoch 9, Loss: 456.6109833253259
Epoch 10, Loss: 454.96377280740177
Epoch 11, Loss: 452.50512268148003
Epoch 12, Loss: 451.8028315335034
Epoch 13, Loss: 451.15240183539566
Epoch 14, Loss: 451.16875762123476
Epoch 15, Loss: 450.541586377149
Epoch 16, Loss: 449.937141047187
Epoch 17, Loss: 449.6132144621987
Epoch 18, Loss: 449.4937028772691
Epoch 19, Loss: 449.40107750714145
Epoch 20, Loss: 449.29037298110717
Epoch 21, Loss: 449.13039712752885
Epoch 22, Loss: 449.14495714973003
Epoch 23, Loss: 449.1546680878828
Epoch 24, Loss: 449.08518965614036
Epoch 25, Loss: 449.1382955867339
Epoch 26, Loss: 449.077481062782
Epoch 27, Loss: 449.14673721395076
Epoch 28, Loss: 449.04735384038423
Epoch 29, Loss: 449.0277063624744
Epoch 30, Loss

In [9]:
def validate(model, data_loader, criterion, device):
    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        for inputs, ages in data_loader:
            inputs, ages = inputs.to(device), ages.to(device).float()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), ages)
            val_running_loss += loss.item()
    return val_running_loss / len(data_loader)


In [10]:
# Validation function
def validate(model, data_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    val_running_loss = 0.0
    with torch.no_grad():  # No gradients needed for validation
        for inputs, ages in data_loader:
            inputs, ages = inputs.to(device), ages.to(device).float()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), ages)
            val_running_loss += loss.item()
    return val_running_loss / len(data_loader)

# Assuming you've already initialized val_loader with the validation dataset
# Compute validation loss
val_loss = validate(model, val_loader, criterion, device)
print(f'Validation Loss: {val_loss}')


Validation Loss: 471.5261697890395


In [11]:
import torch

def rmse(predictions, targets):
    return torch.sqrt(((predictions - targets) ** 2).mean())

def validate_rmse(model, data_loader, device):
    model.eval()
    total_rmse = 0.0
    total_samples = 0

    with torch.no_grad():
        for inputs, ages in data_loader:
            inputs, ages = inputs.to(device), ages.to(device).float()
            outputs = model(inputs).squeeze()  # Ensure outputs are squeezed to match target dimensions
            batch_rmse = rmse(outputs, ages)  # Calculate RMSE for the batch
            total_rmse += batch_rmse.item() * inputs.size(0)  # Accumulate RMSE weighted by batch size
            total_samples += inputs.size(0)

    avg_rmse = total_rmse / total_samples  # Calculate average RMSE across all samples
    print(f'Validation RMSE: {avg_rmse:.4f}')
    return avg_rmse

# Assuming `model`, `val_loader`, and `device` are already defined and initialized
validation_rmse = validate_rmse(model, val_loader, device)


Validation RMSE: 18.6868


In [12]:
def rmse(predictions, targets):
    return torch.sqrt(((predictions - targets) ** 2).mean())


In [13]:
def validate_rmse(model, data_loader, device):
    model.eval()  # Set the model to evaluation mode
    total_rmse = 0.0
    total_samples = 0

    with torch.no_grad():  # No gradients needed for validation
        for inputs, ages in data_loader:
            inputs, ages = inputs.to(device), ages.to(device).float()
            outputs = model(inputs).squeeze()  # Ensure outputs are squeezed to match target dimensions

            # Compute RMSE for the batch and accumulate
            batch_rmse = rmse(outputs, ages)
            total_rmse += batch_rmse.item() * inputs.size(0)  # Multiply by batch size to get total loss
            total_samples += inputs.size(0)

    avg_rmse = total_rmse / total_samples  # Calculate average RMSE across all samples
    print(f'Validation RMSE: {avg_rmse:.4f}')
    return avg_rmse

# Example usage of the validation RMSE function
validation_rmse = validate_rmse(model, val_loader, device)


Validation RMSE: 18.6868


In [14]:
def validate_regression_accuracy(model, data_loader, device, tolerance=7.0):

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, ages in data_loader:
            inputs, ages = inputs.to(device), ages.to(device).float()
            outputs = model(inputs).squeeze()  # Assuming the model outputs a single value per input

            # Determine if each prediction is within the tolerance range of the actual age
            accurate_predictions = torch.abs(outputs - ages) <= tolerance
            correct += accurate_predictions.sum().item()
            total += inputs.size(0)

    accuracy_within_tolerance = 100 * correct / total
    print(f'Validation Accuracy Within ±{tolerance} Years: {accuracy_within_tolerance:.2f}%')
    return accuracy_within_tolerance

# Example usage
val_accuracy_within_tolerance = validate_regression_accuracy(model, val_loader, device)


Validation Accuracy Within ±7.0 Years: 22.41%
